<a href="https://colab.research.google.com/github/AkarisDimitry/PDF-classifier/blob/main/PDF_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Clustering de Documentos de Texto

Este proyecto proporciona un script en Python para realizar clustering no supervisado en datos de texto a partir de archivos PDF y DOC. El script lee y preprocesa los datos de texto, los convierte en una matriz TF-IDF y luego realiza cuatro tipos diferentes de clustering: K-means, jerárquico, Latent Dirichlet Allocation (LDA) y DBSCAN.

In [ ]:
!pip install gensim



In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importando las librerías necesarias
Aquí importamos todas las librerías necesarias para nuestro código. Esto incluye librerías para manipulación de archivos, procesamiento de lenguaje natural, extracción de características, modelado de tópicos y algoritmos de clustering.



In [ ]:
# Importing required libraries
import os
import collections
import matplotlib.pyplot as plt
from PyPDF2 import PdfFileReader
from docx import Document
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from gensim import corpora, models
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram

# Downloading necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')


ModuleNotFoundError: ignored

# Preprocesamiento de datos
En esta sección, definimos las funciones para el preprocesamiento de nuestros datos de texto y la conversión en una matriz TF-IDF.

In [ ]:
# ------------------ Data Preprocessing ------------------

def preprocess_text(text: str) -> str:
    """
    Preprocesses the given text by removing English stopwords and tokenizing.
    """
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return " ".join(filtered_text)

def get_tfidf_matrix(texts: list) -> np.array:
    """
    Converts a list of text documents into a TF-IDF matrix.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    return tfidf_matrix


# Leyendo los archivos
Aquí definimos las funciones para leer archivos PDF y DOC y extraer su contenido de texto.



In [ ]:
# ------------------ File Reading ------------------

def read_pdf_file(file_path: str) -> str:
    """
    Reads the text content from a PDF file.
    """
    with open(file_path, 'rb') as file:
        pdf = PdfFileReader(file)
        text = " ".join(page.extractText() for page in pdf.pages)
    return text

def read_doc_file(file_path: str) -> str:
    """
    Reads the text content from a DOC file.
    """
    doc = Document(file_path)
    text = " ".join(paragraph.text for paragraph in doc.paragraphs)
    return text

def get_text_from_files(directory: str) -> tuple:
    """
    Extracts text from all PDF and DOC files in the given directory and its subdirectories.
    Returns a tuple of two lists: the first list contains the file names,
    and the second list contains the corresponding text content.
    """
    text = []
    names = []
    for dirpath, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.pdf'):
                text += [read_pdf_file(os.path.join(dirpath, filename))]
                names += [dirpath+filename]
            elif filename.endswith('.doc') or filename.endswith('.docx'):
                text += [read_doc_file(os.path.join(dirpath, filename))]
                names += [dirpath+filename]
    return names, text


# **Clustering**
Definimos las funciones para realizar el clustering de nuestros documentos usando varios algoritmos.
## K-means
El algoritmo K-means particiona el conjunto de datos en K grupos distintos y no superpuestos donde cada punto de datos pertenece al grupo con la media más cercana. El algoritmo comienza inicializando K centroides de grupos. Cada punto de datos se asigna entonces al centroide más cercano, y cada centroide se actualiza para ser la media de los puntos de datos asignados a él. Este proceso se repite hasta que las asignaciones ya no cambian o se alcanza un número máximo de iteraciones.

In [ ]:

def kmeans_clustering(tfidf_matrix: np.array, num_clusters: int) -> list:
    """
    Performs K-means clustering on the given TF-IDF matrix.
    """
    km = KMeans(n_clusters=num_clusters)
    km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()
    return clusters

## Clustering Jerárquico
El clustering jerárquico crea un árbol de grupos. El algoritmo comienza tratando cada punto de datos como un solo grupo y luego fusiona los dos grupos que son más similares hasta que solo queda un grupo (o K grupos). El resultado es una representación basada en árboles de los datos (un dendrograma) que muestra el orden en el que se fusionaron los grupos.

In [ ]:
def hierarchical_clustering(tfidf_matrix: np.array, num_clusters: int) -> list:
    """
    Performs hierarchical clustering on the given TF-IDF matrix.
    """
    hc = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean', linkage='ward')
    clusters = hc.fit_predict(tfidf_matrix.toarray())
    return clusters

## Latent Dirichlet Allocation (LDA)
LDA es un tipo de modelo de tópicos probabilístico que supone que cada documento es una mezcla de un cierto número de tópicos y cada palabra en el documento es atribuible a uno de los tópicos del documento. El algoritmo usa un proceso generativo y aplica el teorema de Bayes para inferir la mezcla de tópicos y las asignaciones de tópicos que más probablemente generaron los documentos observados.



In [ ]:
def lda_clustering(preprocessed_documents: list, num_topics: int) -> list:
    """
    Performs LDA (Latent Dirichlet Allocation) to identify topics in the given preprocessed documents.
    """
    texts = [document.split() for document in preprocessed_documents]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    topics = lda.print_topics(num_words=4)
    return topics

## DBSCAN
DBSCAN (Density-Based Spatial Clustering of Applications with Noise) es un algoritmo de clustering basado en la densidad. El algoritmo comienza explorando el conjunto de datos desde un punto aleatorio, y si hay al menos 'min_samples' puntos a una distancia de 'eps' de ese punto, se crea un nuevo grupo. Luego, el algoritmo agrega iterativamente los puntos cercanos a una distancia de 'eps' al grupo. Si un punto no está a una distancia de 'eps' de ningún otro punto, se considera ruido.



In [ ]:
def dbscan_clustering(tfidf_matrix: np.array, eps: float, min_samples: int) -> list:
    """
    Performs DBSCAN clustering on the given TF-IDF matrix.
    """
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(tfidf_matrix.toarray())
    return clusters

# **Visualización**
Finalmente, definimos las funciones para visualizar nuestros resultados de clustering.

In [ ]:
# ------------------ Visualization ------------------

def plot_clusters(tfidf_matrix: np.array, clusters: list) -> None:
    """
    Plots a 2D scatter plot of the given TF-IDF matrix, colored by cluster assignment.
    """
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(tfidf_matrix.toarray())
    plt.scatter(reduced_features[:,0], reduced_features[:,1], c=clusters)
    plt.show()

def plot_dendrogram(model, **kwargs):
    """
    Plots a dendrogram for the given AgglomerativeClustering model.
    """
    children = model.children_
    distance = np.arange(children.shape[0])
    no_of_observations = np.arange(2, children.shape[0]+2)
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)
    dendrogram(linkage_matrix, **kwargs)


# **Script Principal**
Leemos y preprocesamos los documentos, realizamos el clustering y visualizamos los resultados.



In [ ]:
# ------------------ Main Script ------------------

# Read and preprocess documents
names, your_documents = get_text_from_files('/home/akaris/Documents/bibliografia/VDW')
preprocessed_documents = [preprocess_text(doc) for doc in your_documents]
tfidf_matrix = get_tfidf_matrix(preprocessed_documents)

# Perform clustering
kmeans_clusters = kmeans_clustering(tfidf_matrix, num_clusters=3)
hierarchical_clusters = hierarchical_clustering(tfidf_matrix, num_clusters=3)
lda_topics = lda_clustering(preprocessed_documents, num_topics=3)
dbscan_clusters = dbscan_clustering(tfidf_matrix, eps=0.5, min_samples=5)

# Print clustering results
print(names)
print(kmeans_clusters)
print(hierarchical_clusters)
print(lda_topics)
print(dbscan_clusters)

# Plot K-means clusters
plot_clusters(tfidf_matrix, kmeans_clusters)

# Plot hierarchical clusters
hc = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hc = hc.fit(tfidf_matrix.toarray())
plt.title('Hierarchical Clustering Dendrogram')
plot_dendrogram(hc, labels=hc.labels_)
plt.show()
